# Rootz Engine Pipeline
This notebook handles the full pipeline of downloading music from Spotify (searching for equivalent tracks on YouTube), extracting audio, separating stems, and converting bass to MIDI.


# New Section

In [ ]:
# Upgrade pip, wheel, setuptools first
!pip install --upgrade pip wheel setuptools

# Install compatible dependencies explicitly
!pip install numpy==1.26.4 librosa==0.10.1 scikit-learn==1.4.2
!pip install spleeter==2.1.0 youtube_dl mido ffmpeg-python norbert

# Optional: Restart runtime after installation
import os
os.kill(os.getpid(), 9)


  Using cached librosa-0.10.1-py3-none-any.whl.metadata (8.3 kB)
Using cached librosa-0.10.1-py3-none-any.whl (253 kB)
  Attempting uninstall: librosa
    Found existing installation: librosa 0.11.0
    Uninstalling librosa-0.11.0:
      Successfully uninstalled librosa-0.11.0
  Using cached spleeter-2.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached youtube_dl-2021.12.17-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached mido-1.3.3-py3-none-any.whl.metadata (6.4 kB)
  Using cached ffmpeg_python-0.2.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached norbert-0.2.1-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached httpx-0.16.1-py3-none-any.whl.metadata (38 kB)
  Using cached librosa-0.8.0.tar.gz (183 kB)
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.18.5.zip (5.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.

In [ ]:
# Required libraries
!pip install spotipy youtube_dl spleeter mido ffmpeg-python scikit-learn


In [ ]:
# Spotify setup
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '8d76e2b01a0f4285abeefab88d4e1bc3'
client_secret = 'ace6a375e76a4835b361608e632b77ad'
credentials = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=credentials)


In [ ]:
# Get tracks from Spotify playlist
def get_tracks_from_spotify(playlist_url):
    playlist_id = playlist_url.split('/')[-1].split('?')[0]
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return [(track['track']['artists'][0]['name'], track['track']['name']) for track in tracks]

playlist_url = 'https://open.spotify.com/playlist/37i9dQZF1EIUwGkyP8uq62?si=NI4CSkhUStKFJGw9nbHi4g'
tracks = get_tracks_from_spotify(playlist_url)
tracks


In [ ]:
# Search and download from YouTube
import youtube_dl
import os

def download_from_youtube(query, output_dir):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': os.path.join(output_dir, '%(title)s.%(ext)s'),
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([f"ytsearch1:{query}"])

output_dir = '/content/drive/MyDrive/RootzEngine/mp3'
os.makedirs(output_dir, exist_ok=True)

for artist, track in tracks:
    query = f"{artist} - {track}"
    download_from_youtube(query, output_dir)


In [ ]:
# Separate bass stems using Spleeter
!spleeter separate -p spleeter:2stems -o '/content/drive/MyDrive/RootzEngine/stems' '/content/drive/MyDrive/RootzEngine/mp3/*.mp3'


In [ ]:
# Convert bass WAV files to MIDI
# Add your custom conversion script here
